In this notebook, we will calculate features using Trotterization with IBM's QPU.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

/root/workdir/fourier_learning_ibm


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# GPU check
!nvidia-smi -L

GPU 0: NVIDIA A100-PCIE-40GB (UUID: GPU-3fac7503-bb55-35cd-f566-c919eaa55798)
GPU 1: NVIDIA A100-PCIE-40GB (UUID: GPU-bf03abcd-ef36-6733-da57-460c8eb9cab4)
GPU 2: NVIDIA A100-PCIE-40GB (UUID: GPU-bc01f8f3-0939-f2c0-6998-dd49d0360463)
GPU 3: NVIDIA A100-PCIE-40GB (UUID: GPU-b7aeac00-b332-1045-eebc-bb5653ac8402)


In [4]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [5]:
import warnings

warnings.filterwarnings("ignore")

### Config backend

In [6]:
# density matrix simulator
backend_qpu, backend_dm_noiseless, backend_dm_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="density_matrix"
)

print()

# matrix product state simulator
backend_qpu, backend_mps_noiseless, backend_mps_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="matrix_product_state"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['id', 'cz', 'x', 'sx', 'reset', 'measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
  Specific qubit errors: [('id', (0,)), ('id', (1,)), ('id', (2

### Prepare parameters and load dataset

In [7]:
n_qubits = 20

C = 3
n_samples = 55

if n_qubits <= 50:  # n_qubits <= 50
    threshold = 0.6
    n_shots = 1000

    # Load dataset (orient="records")
    if n_qubits == 20:  # 20 量子ビットの場合は、12 量子ビットの結果を使う
        df_n_step = pd.read_json(
            f"results/get_nstep/12Q/threshold{threshold:.2f}/optimal_n_step.json",
            orient="records",
        )
    else:
        df_n_step = pd.read_json(
            f"results/get_nstep/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
            orient="records",
        )
    display(df_n_step)

    # Note that K = n_features-1.
    # n_features counts only the number of real features, also the one of t=0, i.e., constant term.
    # The number of all features, including imaginary numbers, is 2K+1.
    n_features = len(df_n_step)

    times = df_n_step["t"].values
    n_step_array = df_n_step["n_step"].values

else:  # n_qubits > 50
    threshold = None
    n_features = 20
    n_shots = 100
    # 間隔 np.pi/C で等間隔な点を n_features 個生成
    times = np.linspace(0, np.pi / C * (n_features - 1), n_features)

    # n_qubits > 50 の場合は、n_step は 1 から 7 の整数値を取る。
    # まず 1から7までの20個の等間隔の数値を生成
    n_step_array = np.linspace(1, 7, 20)
    # 各値を四捨五入して整数に変換
    n_step_array = [round(x) for x in n_step_array]

print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_shots: {n_shots}")
print(f"n_features: {n_features}")
print(f"times: {times}")
print(f"n_step_array: {n_step_array}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json",
    orient="records",
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

,t,n_step,fidelity
0,0.000000,1,1.000000
1,1.047198,1,0.876665
2,2.094395,1,0.874939
3,3.141593,1,0.865199
4,4.188790,1,0.830213
5,5.235988,2,0.770913
6,6.283185,2,0.762874
7,7.330383,2,0.744273
8,8.377580,2,0.706917
9,9.424778,3,0.670175


n_qubits: 20
C: 3
threshold: 0.6
n_samples: 55
n_shots: 1000
n_features: 12
times: [ 0.          1.04719755  2.0943951   3.14159265  4.1887902   5.23598776
  6.28318531  7.33038286  8.37758041  9.42477796 10.47197551 11.51917306]
n_step_array: [1 1 1 1 1 2 2 2 2 3 3 3]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.0557781201, -0.0124444988, 0.0730114117, 0....",0.639587,NaN,NaN
1,1,"[0.027705067400000002, 0.0543073016, -0.030610...",1.111933,NaN,NaN
2,2,"[-0.0606629037, 0.037721577, -0.01396073820000...",1.144959,NaN,NaN
3,3,"[0.0172949531, -0.0906253368, -0.0969823731, 0...",1.422966,NaN,NaN
4,4,"[-0.051382656000000006, 0.0403074078, 0.014197...",1.258721,NaN,NaN
5,5,"[0.02589247, -0.0274634758, -0.081950795200000...",1.377658,NaN,NaN
6,6,"[-0.0735180061, 0.081111892, 0.0588002475, 0.0...",0.990710,NaN,NaN
7,7,"[-0.0369856516, 0.021958887200000003, -0.11525...",0.994413,NaN,NaN
8,8,"[-0.0711147154, 0.010681738000000001, -0.07737...",1.094811,NaN,NaN
9,9,"[0.0029407147, -0.0476609478, 0.08516097090000...",0.743977,NaN,NaN


# Create Fourier feature

## Trotter simulation

In [8]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_shots": n_shots,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": n_step_array,
    "backend": backend_qpu,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [9]:
# For QPU, we need batch instance and job_ids instead of job instances
batch, _, job_ids = run_job(config, backend_qpu, sim_type="qpu")

In [10]:
path = f"results/fourier_feature_qpu/{n_qubits}Q"

In [11]:
# Save job_ids
with open(f"{path}/job_ids.txt", "w") as f:
    for job_id in job_ids:
        f.write(f"{job_id}\n")

In [13]:
batch.status()

'Closed'

In [14]:
# Load job_ids
with open(f"{path}/job_ids.txt", "r") as f:
    job_ids = [job_id.strip() for job_id in f.readlines()]

# fetch successful jobs
jobs = []
error_count = 0
for i in range(n_samples):
    print(f"Checking job {i}/{n_samples}")
    job_id = job_ids[i]
    job = backend_qpu.service.job(job_id)

    if job.status() == "DONE":
        jobs.append(job)
    else:
        error_count += 1
        print(f"Status: {job.status()}")

Checking job 0/55
Checking job 1/55
Checking job 2/55
Checking job 3/55
Checking job 4/55
Checking job 5/55
Checking job 6/55
Checking job 7/55
Checking job 8/55
Checking job 9/55
Checking job 10/55
Checking job 11/55
Checking job 12/55
Checking job 13/55
Checking job 14/55
Checking job 15/55
Checking job 16/55
Checking job 17/55
Checking job 18/55
Checking job 19/55
Checking job 20/55
Checking job 21/55
Checking job 22/55
Checking job 23/55
Checking job 24/55
Checking job 25/55
Checking job 26/55
Checking job 27/55
Checking job 28/55
Checking job 29/55
Checking job 30/55
Checking job 31/55
Checking job 32/55
Checking job 33/55
Checking job 34/55
Checking job 35/55
Checking job 36/55
Checking job 37/55
Checking job 38/55
Checking job 39/55
Checking job 40/55
Checking job 41/55
Checking job 42/55
Checking job 43/55
Checking job 44/55
Checking job 45/55
Checking job 46/55
Checking job 47/55
Checking job 48/55
Checking job 49/55
Checking job 50/55
Checking job 51/55
Checking job 52/55
Che

In [15]:
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,f_10 Re,f_10 Im,f_11 Re,f_11 Im,expected_value
0,0,0.945,-0.054713,-0.634438,-0.336406,-0.524370,-0.512509,-0.242989,-0.520391,0.003416,...,0.262816,0.288691,0.131904,0.200150,-0.136502,0.060941,-0.239439,-0.082198,-0.239928,0.639587
1,1,0.951,0.558840,-0.368677,0.546603,-0.198811,0.546144,-0.179276,0.496450,-0.072953,...,-0.073288,0.303729,-0.062431,0.246158,-0.068959,0.238003,-0.062727,0.185739,-0.016029,1.111933
2,2,0.946,0.276753,-0.633871,0.422616,-0.472605,0.432984,-0.439115,0.436732,-0.287113,...,-0.089658,0.262964,0.011819,0.202077,-0.050178,0.159316,0.016598,0.129666,0.015127,1.144959
3,3,0.942,0.636237,-0.293797,0.693745,-0.001265,0.599538,0.263568,0.468689,0.406507,...,0.507449,-0.298247,0.440875,-0.354766,0.194242,-0.395464,0.042016,-0.393533,-0.048800,1.422966
4,4,0.949,0.567060,-0.307656,0.586607,-0.224223,0.556815,-0.072160,0.468324,0.126786,...,0.084791,0.083467,0.027246,0.060075,-0.018998,0.105100,-0.078255,0.105151,-0.080301,1.258721
5,5,0.940,0.676981,0.036083,0.547872,0.225594,0.345645,0.337115,0.162118,0.337887,...,0.052933,-0.068625,0.006679,-0.031544,-0.008061,-0.029778,-0.018173,-0.058084,-0.008674,1.377658
6,6,0.946,0.202818,-0.630680,0.186257,-0.569955,0.144575,-0.528942,0.107902,-0.458242,...,-0.210565,0.084156,-0.136157,0.072564,-0.096511,0.103370,-0.090374,0.106988,-0.083443,0.990710
7,7,0.939,0.710293,-0.097306,0.680057,-0.135170,0.683440,-0.077327,0.691019,-0.076669,...,0.070562,0.561678,0.061580,0.446470,0.096238,0.445837,0.182251,0.453484,0.109165,0.994413
8,8,0.934,0.511209,-0.389002,0.570357,-0.279024,0.482119,-0.255933,0.478499,-0.236311,...,-0.092359,0.230107,-0.059588,0.148904,-0.083237,0.110458,-0.081541,0.045620,-0.070943,1.094811
9,9,0.945,0.425321,-0.494208,0.178751,-0.571564,0.057699,-0.573111,-0.110733,-0.452314,...,-0.130320,-0.085435,-0.124903,-0.079759,-0.148164,-0.142214,-0.086025,-0.162389,-0.124072,0.743977
